In [1]:
import import_ipynb

In [3]:
import os
from keras.callbacks import ModelCheckpoint, EarlyStopping
from models import base_model
from utils import load_model, SNR, sum_loss, compile_model
import h5py
from keras.optimizers import Adam


ModuleNotFoundError: No module named 'models'

In [36]:
def load_h5_list(dirname):
    datasets = []
    filenames = os.listdir(dirname)
    file_extensions = set(['.h5'])
    for filename in filenames:
        ext = os.path.splitext(filename)[-1]
        if ext in file_extensions:
            full_filename = os.path.join(dirname, filename)
            datasets.append(full_filename)
    return datasets

In [37]:
def load_h5(dataset_name):
    print('Loading dataset ',dataset_name)
    with h5py.File(dataset_name, 'r') as hf:
        X = (hf['data'][:])
        Y = (hf['label'][:])
    print(X.shape)
    print(Y.shape)
    return X, Y

In [65]:
def compile_model(model, learning_rate):
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='mse', optimizer=optimizer, metrics=[sum_loss, SNR])
    print('model compilation success!')
    return model

In [70]:
def run():
    # Hyper params
    EPOCHS = 60
    BATCH_SIZE = 4
    LOAD_WEIGHTS = True
    WEIGHTS_PATH = 'weights/'
    WEIGHTS_FILE = 'asr-weights.hdf5'
    VALID_SPLIT = 0.05 
    SHUFFLE = True
    MINI_EPOCH = 3 # set each dataset's epochs
    
    model = base_model()
    model = load_model(model, os.path.join(WEIGHTS_PATH, WEIGHTS_FILE), load_weights=LOAD_WEIGHTS)
    model = compile_model(model, 0.0003)
    
    datasets = load_h5_list('/home/joey11121/Audio-SuperRes/data/48K')
    checkpointer = ModelCheckpoint(filepath=os.path.join(WEIGHTS_PATH, WEIGHTS_FILE), verbose=1, save_best_only=True) 
    earlystopper = EarlyStopping(monitor='val_loss', patience=3, verbose=0)
    
    for i in range(EPOCHS):
        print('#REAL EPOCH(%3d/%3d)'%(i+1,EPOCHS))
        for dataset in datasets:
            X,Y = load_h5(dataset)
            model.fit(X, Y,
                    batch_size=BATCH_SIZE,
                    epochs=MINI_EPOCH, 
                    shuffle=SHUFFLE,
                    callbacks=[checkpointer, earlystopper],
                    validation_split=VALID_SPLIT)
    print('Training Finished')

In [71]:
if __name__ == '__main__':
    run()
    #print('Model already trained!')

load base model..
Model: "model_22"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_22 (InputLayer)           (None, 256, 1)       0                                            
__________________________________________________________________________________________________
conv1d_295 (Conv1D)             (None, 128, 16)      272         input_22[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_148 (LeakyReLU)     (None, 128, 16)      0           conv1d_295[0][0]                 
__________________________________________________________________________________________________
conv1d_296 (Conv1D)             (None, 64, 32)       4128        leaky_re_lu_148[0][0]            
_________________________________________________________________________